In [1]:
DATA_NAME = 'standardTesting-full' 
TRANSFORM = 'wavelet-horizontal'
CHANNEL = 'gray'

In [2]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_gray_horizontal_wavelet_full_standardTesting.ipynb


In [3]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [4]:
from scipy import spatial
from sklearn.decomposition import PCA

In [5]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))

In [6]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)
elif 'learned' in TRANSFORM:
    nonskewed_df = pd.read_csv(os.path.join(ROOT_DIR, 'learned-filters', 'nonskewed_filter_idxs_df.csv')).set_index(['dataset', 'num_images', 'num_bootstrap'])
    nonskewed_filter_idxs = eval(nonskewed_df.loc[DATA_NAME].sort_values('num_images', ascending=False)['nonskewed_filter_idxs'].iloc[0]) 
    GROUPS = nonskewed_filter_idxs # can set to filter_group_map.keys() to include all prepared filters

In [7]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [8]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        if len(group_data_map[group]) < 100:
            X[:, i] = np.full(bootstrap_size, np.nan)
        else:
            X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
    cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [9]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df = cov_df.dropna(how='all').dropna(axis=1, how='all')
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

,4,5,6,7,8,9,10
4,1222.33,0.01,0.01,0.00,-0.00,-0.00,0.00
5,0.01,240.97,0.00,0.00,-0.00,-0.00,-0.00
6,0.01,0.00,59.24,0.00,-0.00,0.00,-0.00
7,0.00,0.00,0.00,10.87,0.00,-0.00,-0.00
8,-0.00,-0.00,-0.00,0.00,2.09,-0.00,-0.00
9,-0.00,-0.00,0.00,-0.00,-0.00,0.39,0.00
10,0.00,-0.00,-0.00,-0.00,-0.00,0.00,0.06


In [10]:
corr_matrix = cov_df / np.sqrt(np.outer(np.diag(cov_df), np.diag(cov_df)))
corr_matrix.round(5)

,4,5,6,7,8,9,10
4,1.00000,0.00003,0.00003,0.00001,-0.00001,-0.00003,0.00001
5,0.00003,1.00000,0.00003,0.00003,-0.00001,-0.00005,-0.00004
6,0.00003,0.00003,1.00000,0.00003,-0.00001,0.00001,-0.00001
7,0.00001,0.00003,0.00003,1.00000,0.00002,-0.00001,-0.00005
8,-0.00001,-0.00001,-0.00001,0.00002,1.00000,-0.00003,-0.00002
9,-0.00003,-0.00005,0.00001,-0.00001,-0.00003,1.00000,0.00001
10,0.00001,-0.00004,-0.00001,-0.00005,-0.00002,0.00001,1.00000


In [11]:
np.linalg.norm(cov_df - np.diag(np.diag(cov_df)))  # Remove diagonal elements for visualization

0.023172923452047575

In [12]:
pca = PCA()
pca.fit(cov_df)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[2.13695200e+05 8.07732514e+03 4.67744424e+02 1.48020589e+01
 4.87943273e-01 1.27354103e-02 3.53982414e-34]

Principal components (eigenvectors):


,0,1,2,3,4,5,6
0,0.999385,-0.034095,-0.008021,-0.001468,-0.000283,-0.000053,-0.000008
1,0.033615,0.998019,-0.052389,-0.008936,-0.001717,-0.000320,-0.000051
2,0.009693,0.051588,0.997435,-0.047835,-0.008831,-0.001634,-0.000260
3,0.002200,0.011175,0.046584,0.996555,-0.066603,-0.011768,-0.001871
4,0.000561,0.002845,0.011580,0.064642,0.993132,-0.095698,-0.014455
5,0.000154,0.000779,0.003162,0.017248,0.091234,0.982888,-0.159055
6,0.000050,0.000253,0.001027,0.005602,0.029113,0.156942,0.987162


In [13]:
cos_dist = np.diag(spatial.distance.cdist(eigenvectors, np.eye(cov_df.shape[0]), metric='cosine'))
cos_dist = cos_dist = [np.min(row) for row in cos_dist]
cos_dist


[0.0006147217796529381,
 0.0019806824255582667,
 0.002565381873417927,
 0.003444806304262027,
 0.00686771148705867,
 0.01711152508252134,
 0.012837855438603496]